# Book 6 - Logistic Regression

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import random
import datetime

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
from matplotlib import cm as cm2
matplotlib.style.use('ggplot') 
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, datasets
from sklearn.grid_search import GridSearchCV
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, ExtraTreesClassifier, BaggingRegressor, \
    ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score \
#     confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.cross_validation import train_test_split, KFold, cross_val_score


matplotlib.style.use('ggplot') 
%matplotlib inline

# Bring in X & T train & test

In [2]:
with open('X_train.pkl', 'r') as picklefile:
    X_train = pickle.load(picklefile)
    
with open('X_test.pkl', 'r') as picklefile:
    X_test = pickle.load(picklefile)
    
with open('Y_train.pkl', 'r') as picklefile:
    Y_train = pickle.load(picklefile)
    
with open('Y_test.pkl', 'r') as picklefile:
    Y_test = pickle.load(picklefile)

In [3]:
Y_test = np.transpose(Y_test.values)[0]
Y_train = np.transpose(Y_train.values)[0]
type(Y_test)

numpy.ndarray

In [4]:
with open('Disp_train.pkl', 'r') as picklefile:
    Disp_train = pickle.load(picklefile)
    
with open('Disp_test.pkl', 'r') as picklefile:
    Disp_test = pickle.load(picklefile)

In [5]:
print('X_train length: {0:,}'.format(len(X_train)))
print('X_test  length:  {0:,}'.format(len(X_test)))

X_train length: 23,179
X_test  length:  2,396


In [6]:
X = X_train.append(X_test)
print('Combined length: {0:,}'.format(len(X)))

Combined length: 25,575


In [7]:
X.columns

Index([u'pltfrm', u'state', u'cmpgn_id', u'adv_id', u'period', u'doc_topic',
       u'ad_topic'],
      dtype='object')

In [8]:
start = datetime.datetime.now()

X = pd.get_dummies(X, prefix=['pltfrm', 'state', 'cmpgn_id', 'adv_id',\
                                          'period','doc_topic', 'ad_topic'])

X.drop(['pltfrm_3','state_AK','cmpgn_id_32840','adv_id_1890','period_morning',
       'doc_topic_17.0','ad_topic_91.0'])
finish = datetime.datetime.now()
print(finish - start)

0:00:07.385958


In [9]:
X_train = X[:23179]
X_test = X[23179:]

In [10]:
print('X_train length: {0:,}'.format(len(X_train)))
print('X_test  length:  {0:,}'.format(len(X_test)))

X_train length: 23,179
X_test  length:  2,396


# Logistic Regresion

Log Reg default settings
* correct predictions: 167
* Precent correct: 36.5%

Log Reg default settings (C=10)
* correct predictions: 170
* Precent correct: 37.2%

Log Reg default settings (C=5 & 12)
* correct predictions: 169
* Precent correct: 37.0%


In [79]:
lg = LogisticRegression(C=12)

lg.fit(X_train,Y_train)

LogisticRegression(C=12, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# <font color='blue'> Post Processing Steps

In [80]:
# get probabilities of clicked/not clicked
lg_proba = lg.predict_proba(X_test)
lg_Y_proba = pd.DataFrame(lg_proba,columns = ['p_no_click','p_click'])

In [81]:
ppp = pd.concat([Disp_test, lg_Y_proba], axis=1)
# post processing of predictor

## Reduce to 1-click per display id

In [82]:
checker = ppp
checker.head(3)

,display_id,p_no_click,p_click
0,65537,0.979695,0.020305
1,65537,0.964360,0.035640
2,65537,0.941279,0.058721


In [83]:
checker['Y_test'] = pd.Series(Y_test)
checker['prediction'] = [0 for _ in range(len(checker))] # puts zeros in place
checker.head()

,display_id,p_no_click,p_click,Y_test,prediction
0,65537,0.979695,0.020305,0,0
1,65537,0.964360,0.035640,0,0
2,65537,0.941279,0.058721,0,0
3,65537,0.866812,0.133188,1,0
4,66286,0.751342,0.248658,1,0


In [84]:
# List of unique display_id's, with each display id corresponding to an event
pages = checker['display_id'].unique()
len(pages)

457

In [85]:
# system to use probabilities to choose which page is clicked
# compensates for zero probability across the board, or muliple matching high probabilities

for d_num in pages:  
    indx = 0 # index that becomes click
    disp_list = checker[checker['display_id'] == d_num].index.tolist()

    temp = pd.DataFrame(ppp[ppp['display_id'] == d_num])

    p_max = temp['p_click'].max()

    if p_max == 0:
        indx = random.choice(disp_list) # randomly picks from all matched values
    else:
        counter = 0
        short_list =[]
        for index, row in temp.iterrows():
            if row['p_click'] == p_max:
                counter += 1
                short_list.append(index)
        indx = random.choice(short_list) # randomly picks from all matched values
    checker.set_value(indx, 'prediction', 1)


In [86]:
sum(checker['Y_test'])

457

In [87]:
sum(checker['prediction'])

457

In [78]:
count = 0
for index, row in checker.iterrows():
    if row['prediction'] == 1 and row['Y_test'] == 1:
        count +=1
print('Clicks correctly predicted: {0:,}'.format(count))
print('Percent correctly predicted: {0:%}'.format(float(count)/sum(checker['Y_test'])))

Clicks correctly predicted: 169
Percent correctly predicted: 36.980306%


# <font color='purple'> Gridsearch goes below: <font>

In [89]:
grid = GridSearchCV(lg, scoring = "precision",
                    param_grid = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]})
grid.fit(X_train,Y_train)


/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=12, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='precision', verbose=0)

In [90]:
grid.best_estimator_


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)